## Heat transfer for heat sink

In [ ]:
using Gridap, Gridap.CellData, SparseArrays
using NearestNeighbors, Printf  # make sure this is at the top of your file
using Makie,GLMakie, LaTeXStrings, Makie.GeometryBasics

## 1. Problem
### Let's consider the heat sink as follows:
<img src="imagines/heat_sink.png" style="width:400px;height:350px"/>

The dimensions of the cubic heat sink are $L \times L \times L$ . 

This sink is under a heat flux ($q_{in}$) at the bottom face with $y=0$. The top face is prescribed a temperature of $T = 0 \ ^\circ\mathrm{C}$
 load at $y=L$. The body heat sourse is $s$.


The weak form of this problem is given as follows:

Find $\mathbf{u}$ such that for all test functions $\mathbf{v}$:

$$
a(\mathbf{u}, \mathbf{v}) = l(\mathbf{v}),
$$

where:

$$
a(u,v) = \int_{\Omega} k \nabla v \cdot \nabla u \, d\Omega, 
\quad 
l(v) = -\int_{\Gamma_q} q_{\text{in}} v\, d\Gamma + \int_{\Omega} s v\, d\Omega.
$$

## Inputs

In [16]:
using Gridap

# === Geometry ===
L = 20  # Length of heat sink base
W = 20  # Width and Height
domain = (0, L, 0, W, 0, W)
partition = (20*2, 20*2, 20*2)
model = CartesianDiscreteModel(domain, partition)

Bottom = [1, 2, 5, 6, 9, 11, 17, 18, 23]
Top = [3,4,7,8,10,12,19,20,24]
diff_ = setdiff(1:26, Bottom)

# === Tags for boundary conditions ===
labels = get_face_labeling(model)
add_tag_from_tags!(labels, "bottom", Bottom) # example indices for corners/edges
add_tag_from_tags!(labels, "top", Top) # Top
# === Tag elements at the center of the left face ===
# writevtk(model, "HeatSink")
# === Integration ===
degree = 2
Ω = Triangulation(model)
dΩ = Measure(Ω, degree)
Γ_flux = BoundaryTriangulation(model, tags = "bottom")
dΓ_flux = Measure(Γ_flux, degree)

# === FE space ===
order = 1
reffe = ReferenceFE(lagrangian, Float64, order)

Vₕ = TestFESpace(Ω, reffe; conformity=:H1, dirichlet_tags=["top"])
T₀(x) = VectorValue(0)  # Celsius
Uₕ = TrialFESpace(Vₕ, [T₀])  # Essential BC: T = 0°C on the Dirichlet face



TrialFESpace()

In [17]:
# === Problem data ===
k = 5.0             # W/C
q_in = 20.0          # W/m (heat flux)
s = 6               # W/m^2 Body source term (set to non-zero if needed)

# === Weak form ===
a(u,v) = ∫( k * ∇(v) ⋅ ∇(u) ) * dΩ
l(v) = ∫( -1*q_in * v ) * dΓ_flux + ∫( s * v ) * dΩ

# === Solve ===
op = AffineFEOperator(a, l, Uₕ, Vₕ)
Th = solve(op)

writevtk(Ω, "HeatSink_T", cellfields = ["T" => Th])

(["HeatSink_T.vtu"],)

## Thermal compliance

In [15]:
T = get_free_dof_values(Th) # displacement vector
# Method 1
C1 = sum(∫( k * ∇(Th) ⋅ ∇(Th) ) * dΩ) # Compliance
println("Compliance 1: ", C1)
# Method 2
C2 = sum(∫( -1*q_in * Th ) * dΓ_flux + ∫( s * Th ) * dΩ) # Compliance
println("Compliance 2: ", C2)
# Method 3
u = get_free_dof_values(Th) # displacement vector
K = get_matrix(op) # stiffness matrix
C3 = dot(u, K * u)
println("Compliance 3: ", C3)


Compliance 1: 4.4788e6
Compliance 2: 4.478799999997243e6
Compliance 3: 4.478800000000481e6
